In [ ]:
!pip install transformers datasets evaluate

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
from transformers import AutoTokenizer, AlbertForQuestionAnswering
import torch

In [18]:
tokenizer = AutoTokenizer.from_pretrained("twmkn9/albert-base-v2-squad2")
model = AlbertForQuestionAnswering.from_pretrained("twmkn9/albert-base-v2-squad2")

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
#question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
question, text = "How many programming languages does BLOOM support?", "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."


In [28]:
inputs = tokenizer(question, text, return_tensors="pt")
inputs

{'input_ids': tensor([[    2,   184,   151,  3143,  2556,   630,  8064,   555,    60,     3,
          8064,    63,    13, 11633,  2786, 12905,    17,    92,  7920,  1854,
            19,  5084,  2556,  1112,  2556,    17,   539,  3143,  2556,     9,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [29]:
with torch.no_grad():
    outputs = model(**inputs)

In [30]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)


'13'

In [31]:
# target is "nice puppet"
target_start_index = torch.tensor([12])
target_end_index = torch.tensor([13])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)

9.65